In [1]:
!pip install -q stop-words

import csv
import tensorflow as tf

from tensorflow.keras.utils import pad_sequences

from Helper import *

In [2]:
path = "/tmp/bbc-text.csv"

t = []; l = []

with open(path, "r") as fileread:
  reader = csv.reader(fileread, delimiter=",")
  next(reader)
  for r in reader:
    l.append(r[0])
    t.append(stopper(r[1]))

len(t), len(l)

(2225, 2225)

In [3]:
train_split = 0.8

lt = int(train_split * len(t))

train = t[:lt]; testi = t[lt:]

len(train), len(testi)

(1780, 445)

In [4]:
train_label = l[:lt]; testi_label = l[lt:]

len(train_label), len(testi_label)

(1780, 445)

In [5]:
token = fit_tokener(train, 1000, "<OOV>")

ti = token.word_index

len(ti)

27284

In [6]:
def padded(sentences, token, padding, maxlen):
  i = token.texts_to_sequences(sentences)
  i = pad_sequences(i, maxlen=maxlen, padding=padding)
  return i

train = padded(train, token, "post", 120)

train.shape

(1780, 120)

In [7]:
testi = padded(testi, token, "post", 120)

testi.shape

(445, 120)

In [8]:
import numpy as np

train_l = label_token(l, train_label)

list(train_l[:5])

[array([3]), array([1]), array([0]), array([0]), array([4])]

In [9]:
testi_l = label_token(l, testi_label)

list(testi_l[:5])

[array([4]), array([3]), array([2]), array([0]), array([0])]

In [10]:
from tensorflow.keras import layers

x = layers.Embedding(1000, 16, input_length=120)

i = x(train)

i.shape

TensorShape([1780, 120, 16])

In [11]:
i[0][0].numpy()

array([ 0.04200289,  0.01208641, -0.01187801, -0.02834591, -0.04824691,
        0.03452631, -0.03215799,  0.03466899, -0.03453075, -0.03526612,
        0.02393228, -0.0112816 ,  0.03788682,  0.02059729,  0.0224699 ,
       -0.03688762], dtype=float32)

In [12]:
x = layers.GlobalAveragePooling1D()

i = x(i)

i.shape

TensorShape([1780, 16])

In [13]:
i[0].numpy()

array([ 1.03247128e-02,  2.07103949e-04,  1.72305547e-04,  2.43480280e-02,
        9.20056459e-03, -2.82033434e-04,  1.48089109e-02, -1.97983030e-02,
        5.46928728e-03,  3.41562321e-03, -1.61590776e-03,  1.01777259e-02,
        1.05725685e-02, -5.89051597e-05,  1.55249881e-02,  1.81227122e-02],
      dtype=float32)

In [14]:
x = layers.Dense(24, activation="relu")

i = x(i)

i.shape

TensorShape([1780, 24])

In [15]:
i[0].numpy()

array([0.00340762, 0.00828225, 0.        , 0.00146747, 0.01604233,
       0.02093033, 0.00439863, 0.        , 0.00441225, 0.        ,
       0.02237348, 0.        , 0.        , 0.01000328, 0.003477  ,
       0.00118908, 0.00798261, 0.01728036, 0.        , 0.01330486,
       0.        , 0.        , 0.        , 0.        ], dtype=float32)

In [16]:
x = layers.Dense(5, activation="softmax")

i = x(i)

i.shape

TensorShape([1780, 5])

In [17]:
i[0].numpy()

array([0.20206475, 0.1989861 , 0.1974947 , 0.20081858, 0.20063578],
      dtype=float32)

In [ ]:
model = tf.keras.models.Sequential([
    layers.Embedding(1000, 16, input_length=120),
    layers.GlobalAveragePooling1D(),
    layers.Dense(24, activation="relu"),
    layers.Dense(5, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

hist = model.fit(train, train_l, validation_data=(testi, testi_l), epochs=10)